In [17]:
# Jai Mata Di
# Project on DataScience
# Author : Aditya Upadhyay
# Date : 2024 - 10 - 29
# Proffessor Supratim Shit


In [ ]:
import pandas as pd
from scipy.stats import shapiro

# Load your dataset
data = pd.read_csv(r"archive (2)\city_hour.csv")

# Option 1: Impute missing values with the mean
# data['AQI'].fillna(data['AQI'].mean(), inplace=True)

# Option 2: Impute missing values with the median (uncomment if preferred)
data['AQI'].fillna(data['AQI'].median(), inplace=True)

# Perform Shapiro-Wilk test on the AQI column
stat, p_value = shapiro(data['AQI'])

# Print results
alpha = 0.05  # significance level
if p_value > alpha:
    print("AQI column appears to follow a normal distribution (fail to reject H0).")
else:
    print("AQI column does not follow a normal distribution (reject H0).")


AQI column does not follow a normal distribution (reject H0).


C:\Users\harsh\AppData\Local\Temp\ipykernel_33308\3802103883.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['AQI'].fillna(data['AQI'].median(), inplace=True)
C:\Users\harsh\AppData\Local\Temp\ipykernel_33308\3802103883.py:14: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 707875.
  stat, p_value = shapiro(data['AQI'])


In [19]:
import pandas as pd
from scipy.stats import f_oneway


# Group AQI data by city
cities = data['City'].unique()
aqi_groups = [data[data['City'] == city]['AQI'] for city in cities]

# Perform ANOVA
stat, p_value = f_oneway(*aqi_groups)

# Print results
alpha = 0.05  # significance level
if p_value > alpha:
    print("No significant difference in AQI among cities (fail to reject H0).")
else:
    print("Significant difference in AQI among cities (reject H0).")


Significant difference in AQI among cities (reject H0).


In [20]:
from scipy.stats import zscore, f_oneway
import pandas as pd


# Fill null values with the median only for numerical columns
numerical_cols = data.select_dtypes(include=['number']).columns
data[numerical_cols] = data[numerical_cols].apply(lambda x: x.fillna(x.median()))

# Select only the dependent columns (PM2.5, NO, NO2, CO, AQI) for analysis
data_dependent = data[['PM2.5', 'NO', 'NO2', 'CO', 'AQI']]

# Normalize each column using z-score normalization
data_normalized = data_dependent.apply(zscore)

# Organize each normalized column as a separate group for ANOVA
groups = [data_normalized[col] for col in data_normalized.columns]

# Perform ANOVA test on the dependent columns
stat, p_value = f_oneway(*groups)

# Print results
alpha = 0.05  # significance level
if p_value > alpha:
    print("No significant difference in means among normalized PM2.5, NO, NO2, CO, and AQI (fail to reject H0).")
else:
    print("Significant difference in means among normalized PM2.5, NO, NO2, CO, and AQI (reject H0).")

# Display p-value for further analysis
print(f"ANOVA test p-value: {p_value}")


No significant difference in means among normalized PM2.5, NO, NO2, CO, and AQI (fail to reject H0).
ANOVA test p-value: 1.0


In [21]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, chi2

# Load the dataset
# data = pd.read_csv('your_dataset.csv')  # Uncomment and specify your dataset path

# Select only columns with chemical attributes
chemical_columns = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene']

# Impute missing values with the median for each chemical attribute column
data[chemical_columns] = data[chemical_columns].apply(lambda x: x.fillna(x.median()), axis=0)

# Bin each chemical attribute into 4 categories
for col in chemical_columns:
    data[col] = pd.qcut(data[col], q=4, labels=False, duplicates='drop')

# Degrees of freedom for a 4x4 contingency table
df = (4 - 1) * (4 - 1)

# Bonferroni correction for multiple comparisons
alpha = 0.05
num_comparisons = len(chemical_columns) * (len(chemical_columns) - 1) / 2
adjusted_alpha = alpha / num_comparisons
adjusted_critical_value = chi2.ppf(1 - adjusted_alpha, df)

# Print adjusted critical value and hypotheses
print(f"Adjusted Critical Value for Chi-square test (df={df}, adjusted alpha={adjusted_alpha:.5f}): {adjusted_critical_value:.2f}")
print("Null Hypothesis (H0): There is no association between the two categorical variables.")
print("Alternative Hypothesis (H1): There is an association between the two categorical variables.")
print("\nInterpretation: If the test statistic is greater than the adjusted critical value, we reject the null hypothesis (indicating a significant association).\n")

# Initialize a DataFrame to store Chi-square test statistics for each chemical pair
chi_square_matrix = pd.DataFrame(index=chemical_columns, columns=chemical_columns)
dependent_pairs = []

# Calculate Chi-square statistic for each pair
for i, col1 in enumerate(chemical_columns):
    for j, col2 in enumerate(chemical_columns):
        if i == j:
            chi_square_matrix.loc[col1, col2] = np.nan  # Diagonal set to NaN
        elif j > i:  # Check each pair only once
            # Create a contingency table
            contingency_table = pd.crosstab(data[col1], data[col2])

            # Perform the Chi-square test
            chi2_stat, _, _, _ = chi2_contingency(contingency_table)

            # Store the Chi-square statistic in the matrix
            chi_square_matrix.loc[col1, col2] = f"{chi2_stat:.2f}"
            chi_square_matrix.loc[col2, col1] = f"{chi2_stat:.2f}"  # Fill symmetric cell

            # Check if the test statistic indicates dependency with adjusted critical value
            if chi2_stat > adjusted_critical_value:
                dependent_pairs.append((col1, col2))

# Display the Chi-square statistics matrix
print("\nChi-square Test Statistic Matrix:")
print(chi_square_matrix)

# Print pairs of dependent variables
print("\nDependent Variable Pairs (where H0 is rejected after Bonferroni correction):")
for pair in dependent_pairs:
    print(f"{pair[0]} and {pair[1]}")

print(len(dependent_pairs))

Adjusted Critical Value for Chi-square test (df=9, adjusted alpha=0.00076): 28.60
Null Hypothesis (H0): There is no association between the two categorical variables.
Alternative Hypothesis (H1): There is an association between the two categorical variables.

Interpretation: If the test statistic is greater than the adjusted critical value, we reject the null hypothesis (indicating a significant association).


Chi-square Test Statistic Matrix:
             PM2.5       PM10         NO        NO2        NOx        NH3  \
PM2.5          NaN  380263.96  145051.12  187376.34  128767.35   99617.03   
PM10     380263.96        NaN  137511.34  157513.59  126663.29  225519.83   
NO       145051.12  137511.34        NaN  211100.66  437468.83  104718.82   
NO2      187376.34  157513.59  211100.66        NaN  352460.42  154606.37   
NOx      128767.35  126663.29  437468.83  352460.42        NaN   62653.34   
NH3       99617.03  225519.83  104718.82  154606.37   62653.34        NaN   
CO       101

In [22]:
import pandas as pd

# Load your dataset


# Get unique values in the 'AQI_Bucket' column
unique_aqi_buckets = data['AQI_Bucket'].unique()

# Print unique AQI bucket values
print("Unique values in 'AQI_Bucket':", unique_aqi_buckets)


Unique values in 'AQI_Bucket': [nan 'Poor' 'Moderate' 'Very Poor' 'Severe' 'Satisfactory' 'Good']


In [23]:
import pandas as pd
from scipy.stats import chisquare


data['AQI_Bucket'] = data['AQI_Bucket'].fillna(data['AQI_Bucket'].mode()[0])

# Count the occurrences of each category in 'AQI Bucket'
observed_counts = data['AQI_Bucket'].value_counts()

# Define expected counts (for a uniform distribution, you can set them equal)
total_counts = observed_counts.sum()
num_categories = len(observed_counts)
expected_counts = [total_counts / num_categories] * num_categories  # Uniform expectation

# Perform Chi-square goodness-of-fit test
chi_stat, p_value = chisquare(f_obs=observed_counts, f_exp=expected_counts)

# Print the results
print(f"Chi-square Statistic: {chi_stat:.2f}, p-value: {p_value:.4f}")
print("Observed Counts:")
print(observed_counts)
print("Expected Counts:")
e=[int(i) for i in expected_counts]
print(e)

# Conclusion based on p-value
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in proportions among the categories.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in proportions among the categories.")


Chi-square Statistic: 593329.19, p-value: 0.0000
Observed Counts:
AQI_Bucket
Moderate        328071
Satisfactory    189434
Poor             66654
Very Poor        57455
Good             38611
Severe           27650
Name: count, dtype: int64
Expected Counts:
[117979, 117979, 117979, 117979, 117979, 117979]
Reject the null hypothesis: There is a significant difference in proportions among the categories.
